# A mobile app for lottery addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

### Core Functions

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, which means once a number is drawn, it's not put back in the set.

We'll use two principal function for calculate `factorial` and `combinations`.

In [3]:
def factorial(n):
    """
    Calculate the factorial of n (n!)
    Args:
        n: numeric value
    Returns:
        The n!.
    Examples
    ________
    >>> factorial(5)
    120
    """
    if n == 2:
        return 2
    return n * factorial(n - 1)

In [4]:
def combinations(n, k):
    """
    Calculate the combinations of n taking k objects.
    Args:
        n: number represent a group.
        k: number of takin objectcs from group.
    Returns:
        The nCk.
    Examples
    ________
    >>> combinations(52, 5)
    2598960.0
    """
    return factorial(n) / (factorial(k) * factorial(n - k))


##  One-ticket Probability
We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to our function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [27]:
def one_ticket_probability(numbers):
    outcomes = combinations(49, 6)
    successful_outcome = 1 / outcomes
    percentage_win = successful_outcome * 100
    print(f'The probability of the numbers {numbers} winning the grand prize is {percentage_win} % (1 / {int(outcomes)} possible combinations)')    

In [28]:
one_ticket_probability(['13', '17', '19', '10', '30', '43'])

The probability of the numbers ['13', '17', '19', '10', '30', '43'] winning the grand prize is 7.151123842018516e-06 % (1 / 13983816 possible combinations)


In [29]:
one_ticket_probability(['3', '1', '4', '1', '6', '5'])

The probability of the numbers ['3', '1', '4', '1', '6', '5'] winning the grand prize is 7.151123842018516e-06 % (1 / 13983816 possible combinations)


In [31]:
import pandas as pd
lottery = pd.read_csv('649.csv')

In [32]:
lottery.shape

(3665, 11)

In [33]:
lottery.head()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34
3,649,4,0,7/3/1982,3,9,10,13,20,43,34
4,649,5,0,7/10/1982,5,14,21,31,34,47,45


In [35]:
lottery.tail()

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3660,649,3587,0,6/6/2018,10,15,23,38,40,41,35
3661,649,3588,0,6/9/2018,19,25,31,36,46,47,26
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17
